# 509 Final Project

This notebook reads in a CSV file that contain an attribute with raw HTML, then parses the article text using Beautiful Soup. A copy of the full DF is written to a CSV file for sharing with all colloborators in GitHub.

## Globally import libraries

In [1]:
import numpy as np
import pandas as pd
import pymysql as mysql
import matplotlib.pyplot as plt
import os
import shutil
import re
import logging
import time
import datetime as dt
import zipfile
import requests
from bs4 import BeautifulSoup
import datetime
import re
import regex as rex
from collections import defaultdict, Counter
import random
import json
#import mysql.connector

# Set pandas global options
pd.options.display.max_rows = 17

## Upload data from CSV

In [2]:
'''Dir nav citation:
https://softhints.com/python-change-directory-parent/'''
curr_dir = os.path.abspath(os.curdir)
print(curr_dir)
os.chdir("..")
up1_dir = os.path.abspath(os.curdir)
print(up1_dir)

C:\Users\acarr\Documents\GitHub\ADS509_Final_project\deliverables
C:\Users\acarr\Documents\GitHub\ADS509_Final_project


In [3]:
# change `data_location` to the location of the folder on your machine.
data_r_location = 'data_restricted'
data_location = 'data'

file_in_name = 'data_raw_amc.csv'
file_out_name = 'data_parsed_amc.csv'

file_in_path01 = os.path.join(up1_dir, data_r_location, file_in_name)
file_out_path = os.path.join(up1_dir, data_location, file_out_name)

print(f'CSV file in path: {file_in_path01}')
print(f'CSV file out path: {file_out_path}')

CSV file in path: C:\Users\acarr\Documents\GitHub\ADS509_Final_project\data_restricted\data_raw_amc.csv
CSV file out path: C:\Users\acarr\Documents\GitHub\ADS509_Final_project\data\data_parsed_amc.csv


In [4]:
slct_tbl_full_df01 = pd.read_csv(file_in_path01)
display(slct_tbl_full_df01.head())

,text_id,source_name,author,title,url,publish_date,article_text,content
0,1,CNN,"Clare Foran,Nicky Robertson",Senate races to avert default but vote timing ...,https://www.cnn.com/2023/06/01/politics/senate...,2023-06-01T09:00:40Z,"<!DOCTYPE html>\n<html data-layout-uri=""cms.cn...",NaN
1,2,Fox News,Paul Steinhauser,First on Fox: Pro-Tim Scott super PAC launches...,https://www.foxnews.com/politics/pro-scott-sup...,2023-06-01T16:12:56Z,"<!DOCTYPE html>\n<html data-n-head=""%7B%22lang...",NaN
2,3,Fox News,Greg Wehner,Pennsylvania bus driver allegedly used duct ta...,https://www.foxnews.com/us/pennsylvania-bus-dr...,2023-05-31T00:21:20Z,"<!DOCTYPE html>\n<html data-n-head=""%7B%22lang...",NaN
3,4,Fox News,Michael Ruiz,Bob Lee murder: Cash App founder seen with sus...,https://www.foxnews.com/us/bob-lee-murder-cash...,2023-05-31T20:30:12Z,"<!DOCTYPE html>\n<html data-n-head=""%7B%22lang...",NaN
4,5,Fox News,Brooke Singman,Senate GOP demands answers on security clearan...,https://www.foxnews.com/politics/senate-gop-de...,2023-05-31T22:11:38Z,"<!DOCTYPE html>\n<html data-n-head=""%7B%22lang...",NaN


## Extract article data

### Check for missing articles

In [5]:
count_nan = slct_tbl_full_df01['article_text'].isnull().sum()
 
# printing the number of values present
# in the column
print('Number of NaN values present: ' + str(count_nan))

Number of NaN values present: 0


### Parse the article text from the column with raw HTML

In [6]:
slct_tbl_full_df02 = slct_tbl_full_df01.copy()
#slct_tbl_full_df02 = slct_tbl_full_df01.sort_values(by=['source_name'])
#slct_tbl_full_df02 = slct_tbl_full_df02.reset_index()

print(f'DF instances: {len(slct_tbl_full_df02)}')

slct_tbl_full_df02['article_parsed'] = ''

total_urls = len(slct_tbl_full_df02)

# Start timer
start_time = dt.datetime.today()

# Use multiple parsing methods to extract the article from raw HTML
for i, row in enumerate(slct_tbl_full_df02.itertuples(), 1):
    #print(f'Enumeration #: {i}')
    #print(row[7])
    soup = BeautifulSoup(row[7], 'html.parser')

    # Check for available JSON object
    try:
        script_tag = soup.find('script', {'type': 'application/ld+json'})
        if script_tag == None:
            json_err01 = f'''JSON Object = None: Index: {i-1}; source: {row[2]};
            URL: {row[5]}'''
        else:
            article_json = json.loads(script_tag.string)
            article_content = article_json['articleBody']
            slct_tbl_full_df02.at[row.Index, 'article_parsed'] = article_content

    # If no JSON object available, use Beautiful Soup to look for available 
    # HTML tags
    except TypeError:
        print(f'Type Error')
    
    except KeyError:
        json_err02 = f'''Missing JSON key: Index: {i-1}; source: {row[2]};
        URL: {row[5]}'''
        article_body = soup.find('div', class_='article__content-container')

        if article_body is None: #forfoxandbreitbert(sometimes)
            #print('Class != article__content-container')
            article_body = soup.find('p', class_="speakable")
            if article_body is None: #breitbert(most)
                #print('Class != speakable')
                article_body = soup.find('div', class_='entry-content')
                if article_body is None: #WashPost
                    #print('Class != entry-content')
                    article_body = soup.find('div',class_='article-body')

        if article_body is not None:
            article_text = article_body.get_text()
            slct_tbl_full_df02.at[row.Index, 'article_parsed'] = article_text
            #print('Rejoice, parse was successful!')
        else:
            print('\nParse not successful...')
            try:
                print(json_err02)
            except:
                pass

    print('.', end='')
        
# End timer script
end_time = dt.datetime.today()
time_elapse = end_time - start_time
print(f'Start Time = {start_time}')
print(f'End Time = {end_time}')
print(f'Elapsed Time = {time_elapse}')

DF instances: 6153
................................................................
Parse not successful...
Missing JSON key: Index: 64; source: CNN;
        URL: https://www.cnn.com/politics/live-news/us-debt-ceiling-deadline-talks-05-31-23/index.html
.......................
Parse not successful...
Missing JSON key: Index: 87; source: CNN;
        URL: https://www.cnn.com/politics/live-news/us-debt-ceiling-senate-vote-06-01-23/index.html
................................................................................................................................................................................................................................................................................................................................
Parse not successful...
Missing JSON key: Index: 407; source: CNN;
        URL: https://www.cnn.com/politics/live-news/us-debt-ceiling-deadline-talks-05-30-23/index.html
..................................................................

In [7]:
# Review sample
display(slct_tbl_full_df02.iloc[6119])
#display(slct_tbl_full_df02.iloc[6119]['article_text'])

text_id                                                        8413
source_name                                          Slate Magazine
author                                                 Nitish Pahwa
title             Wall Street’s Newest Darling Is an Unprofitabl...
url               https://slate.com/business/2023/06/cava-stock-...
publish_date                                   2023-06-16T22:22:57Z
article_text      <!DOCTYPE html>\n<html data-env="prod" data-la...
content           Stock markets aint what they used to be. Since...
article_parsed                                                     
Name: 6119, dtype: object

In [8]:
display(slct_tbl_full_df02[slct_tbl_full_df02['article_parsed']==''].head(11))
print(len(slct_tbl_full_df02[slct_tbl_full_df02['article_parsed']=='']))

,text_id,source_name,author,title,url,publish_date,article_text,content,article_parsed
64,65,CNN,"By <a href=""/profiles/adrienne-vogt"">Adrienne ...",The latest on the US debt ceiling deal,https://www.cnn.com/politics/live-news/us-debt...,2023-05-31T12:35:50Z,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",NaN,
74,75,Fox News,Sofie Watson,FOX NEWS CHANNEL TO DEBUT NEW WEEKEND PRIMETIM...,https://press.foxnews.com/2023/06/fox-news-cha...,2023-06-01T18:33:43Z,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",NaN,
87,88,CNN,"By <a href=""/profiles/maureen-chowdhury"">Maure...",The latest on the US debt ceiling deal,https://www.cnn.com/politics/live-news/us-debt...,2023-06-01T13:46:13Z,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",NaN,
255,257,CNN,NaN,UPDATE: WAR ON GAY PRIDE...,https://view.newsletters.cnn.com/messages/1685...,2023-06-01T21:55:37Z,"<!DOCTYPE html>\n<html lang=""en"" xmlns:o=""urn:...",NaN,
407,566,CNN,"By Mike Hayes and <a href=""/profiles/maureen-c...",The latest on the US debt ceiling deal,https://www.cnn.com/politics/live-news/us-debt...,2023-05-30T14:12:09Z,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",NaN,
541,760,Fox News,Danielle Tuntigian,SEN. JONI ERNST AND SEN. JEANNE SHAHEEN TO PAR...,https://press.foxnews.com/2023/05/sen-joni-ern...,2023-05-30T17:07:24Z,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",NaN,
665,1173,CNN,By Sophie Tanno,Russia's war in Ukraine,https://www.cnn.com/europe/live-news/russia-uk...,2023-05-06T09:46:06Z,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",Russia has been thwarting US-made mobile rocke...,
948,1456,The Washington Post,Adriana Usero,"Proud Boys revealed: Videos, secret chats show...",https://www.washingtonpost.com/dc-md-va/intera...,2023-05-05T16:00:23Z,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",The man on the left in the video below alleged...,
1122,1737,CNN,"By <a href=""/profiles/aditi-sandal"">Aditi Sang...",8 killed when driver plows into crowd outside ...,https://www.cnn.com/us/live-news/brownsville-t...,2023-05-08T14:28:31Z,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",Editors Note: This story contains graphic desc...,
1341,2025,CNN,"By <a href=""/profiles/adrienne-vogt"">Adrienne ...",George Santos charged in federal probe,https://www.cnn.com/politics/live-news/george-...,2023-05-10T12:52:24Z,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...","Rep. George Santos, the freshman congressman w...",


75


In [9]:
count_p_nan = slct_tbl_full_df02['article_parsed'].isnull().sum()
 
# printing the number of values present
# in the column
print('Number of NaN values present: ' + str(count_p_nan))

Number of NaN values present: 0


### Write datafrme to CSV

In [10]:
slct_tbl_full_df03 = slct_tbl_full_df02[['source_name',
                                         'author',
                                         'title',
                                         'url',
                                         'publish_date',
                                         'article_parsed']]

slct_tbl_full_df03.to_csv(file_out_path, index=False)